In [ ]:
#vcf input file
vcffile="final.recode.vcf"
skipr=5211  #count header lines in command line (grep -c "##" batch_1.vcf)
skipc=9  #non sample columns

#samples "database"
samplefile="/home/megan/megan/research/eucalyptus/eucalyptus_data/Emelliodora_PlantsSamples.csv"
accessionfile="/home/megan/megan/research/eucalyptus/eucalyptus_data/Emelliodora_Accessions.csv"

In [ ]:
%load_ext rpy2.ipython

# read in metadata

In [ ]:
%%script env vcffile="$vcffile" bash
#get sample list
grep "^#CHROM" $vcffile | sed 's/#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\t//'g > samples.txt

In [ ]:
%%R -i samplefile,accessionfile -o samples

#get sample list
sample_list=scan("samples.txt", what="character", sep="\t")

#add sample metadata
sampleinfo=read.csv(samplefile, header=T)  #read in sample information
samplematches=match(sampleinfo$SampleID, sample_list)  #subset for just samples in this analysis
samples=sampleinfo[!is.na(samplematches),][order(na.omit(samplematches)),]

accessioninfo=read.csv(accessionfile, header=T)  #read in accession information
accessionmatches=match(samples$AccessionID, accessioninfo$AccessionID)
samples=cbind(samples,accessioninfo$Latitude[accessionmatches],accessioninfo$Longitude[accessionmatches],
              accessioninfo$PopulationName[accessionmatches])
names(samples)[names(samples)=="accessioninfo$Latitude[accessionmatches]"]="Latitude"
names(samples)[names(samples)=="accessioninfo$Longitude[accessionmatches]"]="Longitude"
names(samples)[names(samples)=="accessioninfo$PopulationName[accessionmatches]"]="PopulationName"
samples$PopulationName=droplevels(samples$PopulationName) #fix annoying extra level with no assignment

# check for outliers

In [ ]:
%%R -i vcffile,samples,skipr,skipc -o genos.gi

library(adegenet)

#read in vcf, skipping ##header rows and removing other junk
vcf=read.delim(vcffile, sep="\t", header=T, skip=skipr)
vcf=vcf[c(-skipc:-1)] #remove non sample columns
genos=apply(vcf,2,substr,1,3)  #get just genotypes
genos=t(genos)  #transpose 
#row.names(genos)=substr(row.names(genos),9,14)  #get just sample ID
genos[1:5,1:10]  #print bit to check its ok 
dim(genos)  #print number of samples and loci
genos.filt=genos
#format for input into genid object
genos.filt[genos.filt == "./."] <- NA
#convert to genid object
genos.gi <- df2genind(genos.filt,sep="/",NA.char=NA)
genos.gi

In [ ]:
%%R -i samples -o genos.gi

#preliminary pca of genetic distance (to find additional outliers)
genos.dist=dist(genos.gi)
pcoa.genos=dudi.pco(genos.dist,scannf = F, nf = 3)
percent_var=round(100*pcoa.genos$eig/sum(pcoa.genos$eig),1)
#pdf("emellb_pca_280.pdf")
plot(pcoa.genos$li[,1:2], pch="",
   xlab = paste("PCOA axis 1 (", percent_var[1],"%)"), ylab = paste("PCOA axis 2 (", percent_var[2],"%)"))
  #  xlim=c(-10,10),ylim=c(-15,10))
text(pcoa.genos$li[,1:2], 
     label=samples$SampleName, cex = .6) #add names
#dev.off()

# summarize data (with outliers removed)

In [ ]:
%%script env vcffile="$vcffile" bash

### PUT SAMPLE ID's HERE FOR OUTLIERS FOR REMOVAL ###
remove="--remove-indv S45093 --remove-indv S45095 --remove-indv S45193 --remove-indv S45194 --remove-indv S45195"

vcftools --vcf $vcffile $remove --depth
vcftools --vcf $vcffile $remove --site-depth
vcftools --vcf $vcffile $remove --site-mean-depth
vcftools --vcf $vcffile $remove --missing-indv
vcftools --vcf $vcffile $remove --missing-site
vcftools --vcf $vcffile $remove --het
vcftools --vcf $vcffile $remove --hardy
vcftools --vcf $vcffile $remove --freq
awk '{print $6}' out.frq | awk -F":" '{print $2}' > out.maf #generate maf file

In [ ]:
%%R

#look at samples
idepth=read.delim("out.idepth")
hist(idepth$MEAN_DEPTH, main="mean locus depth per individual", xlab="mean depth")
imiss=read.delim("out.imiss")
hist(imiss$F_MISS, main="missingness per individual", xlab="proportion missing")
plot(1-imiss$F_MISS, idepth$MEAN_DEPTH, main="by individual", 
     xlab="proportion loci genotyped", ylab="mean depth per locus")

#sample heterozygosity
heteroz=read.delim("out.het")
hist(heteroz$F, main="sample inbreeding coefficient (F)", xlab="F")

#look at loci
ldepthtot=read.delim("out.ldepth")
ldepth=read.delim("out.ldepth.mean")
hist(ldepth$MEAN_DEPTH, main="mean individual depth per locus", xlab="mean depth")
lmiss=read.delim("out.lmiss")
hist(lmiss$F_MISS, main="missingness per locus", xlab="proportion missing")
plot(1-lmiss$F_MISS, ldepthtot$SUM_DEPTH, main="by locus", 
     xlab="proportion individuals genotyped", ylab="total locus depth")

#maf
maf=scan("out.maf")
hist(maf,breaks=100, main="alternate allele frequency", xlab="allele frequency")

#heterozygosity, hwe
hwe=read.delim("out.hwe")
plot(maf,hwe$P_HWE, xlab="alternate allele frequency", ylab="HWE (p-value)")
hist(hwe$P_HWE, main="HWE", xlab="p-value")
hist(hwe$P_HET_DEFICIT, main="heterozygote deficit", xlab="p-value")
hist(hwe$P_HET_EXCESS, main="heterozygote excess", xlab="p-value")
hwe=read.delim("out.hwe")
obs_hwe=data.frame(do.call('rbind', strsplit(as.character(hwe$OBS.HOM1.HET.HOM2.),'/',fixed=TRUE)))
obs_hwe=cbind(hwe$CHR,hwe$POS,obs_hwe)
colnames(obs_hwe)=c("CHR","POS","homo1","hets","homo2")
obs_hwe$homo1=as.numeric(as.character(obs_hwe$homo1))
obs_hwe$hets=as.numeric(as.character(obs_hwe$hets))
obs_hwe$homo2=as.numeric(as.character(obs_hwe$homo2))
obs_hwe$prop_homo1=as.numeric(as.character(obs_hwe$homo1))/(obs_hwe$homo1+obs_hwe$hets+obs_hwe$homo2)
obs_hwe$prop_hets=as.numeric(as.character(obs_hwe$hets))/(obs_hwe$homo1+obs_hwe$hets+obs_hwe$homo2)
obs_hwe$prop_homo2=as.numeric(as.character(obs_hwe$homo2))/(obs_hwe$homo1+obs_hwe$hets+obs_hwe$homo2)
hist(obs_hwe$prop_homo1, main="proportion of homozygous for reference allele", xlab="proportion")
hist(obs_hwe$prop_hets, main="proportion of heterozygotes", xlab="proportion")
hist(obs_hwe$prop_homo2, main="proportion of homozygous for alternate allele", xlab="proportion")
exp_hwe=data.frame(do.call('rbind', strsplit(as.character(hwe$E.HOM1.HET.HOM2.),'/',fixed=TRUE)))
colnames(exp_hwe)=c("homo1","hets","homo2")
exp_hwe$hets=as.numeric(as.character(exp_hwe$hets))
f=(exp_hwe$hets-obs_hwe$hets)/(exp_hwe$hets)
hist(f, main="locus F ((exp-obs)/exp)")

#matrix of counts of homozygotes (ref/alt) and heterozygotes
homohet_sum=matrix(data=NA, nrow=4, ncol=3)
rownames(homohet_sum)=c("none","one","two or less","more than two")
colnames(homohet_sum)=c("homo_ref","het","homo_alt")
homohet_sum[1,1]=sum(obs_hwe$homo1==0)
homohet_sum[1,2]=sum(obs_hwe$hets==0)
homohet_sum[1,3]=sum(obs_hwe$homo2==0)
homohet_sum[2,1]=sum(obs_hwe$homo1==1)
homohet_sum[2,2]=sum(obs_hwe$hets==1)
homohet_sum[2,3]=sum(obs_hwe$homo2==1)
homohet_sum[3,1]=sum(obs_hwe$homo1<=2)
homohet_sum[3,2]=sum(obs_hwe$hets<=2)
homohet_sum[3,3]=sum(obs_hwe$homo2<=2)
homohet_sum[4,1]=sum(obs_hwe$homo1>2)
homohet_sum[4,2]=sum(obs_hwe$hets>2)
homohet_sum[4,3]=sum(obs_hwe$homo2>2)
print(homohet_sum)

# format genotype matrix, baypass

In [ ]:
%%R -i vcffile,skipr,skipc,samples -o genos

library(stringr)

#read in vcf, skipping ##header rows and removing other junk
vcf=read.delim(vcffile, sep="\t", header=T, skip=skipr)
loci_names=paste(vcf[,1],"_",vcf[,2], sep="")
vcf=vcf[c(-skipc:-1)] #remove non sample columns
genos=apply(vcf,2,substr,1,3)  #get just genotypes
genos=t(genos)  #transpose 
#row.names(genos)=substr(row.names(genos),9,14)  #get just sample ID
colnames(genos)=loci_names
#print(genos[1:3,1:5])  #print bit to check its ok 
print(dim(genos))  #print number of samples and loci

#remove outliers
genos.filt=genos
### PUT SAMPLE ID's HERE FOR OUTLIERS FOR REMOVAL ###
remove_ids=c("S45093","S45095","S45193","S45194","S45195")
genos.filt=genos.filt[!rownames(genos.filt) %in% remove_ids,]
samples=samples[!samples$SampleID %in% remove_ids,]
print(dim(samples))

#put in matrix format
#0=homoz alt allele #1=heterozygote #2=homoz ref allele
genos.matrix=genos.filt
genos.matrix[genos.matrix == "./."] <- "NA"
genos.matrix[genos.matrix == "1/1"] <- "0"
genos.matrix[genos.matrix == "0/1"] <- "1"
genos.matrix[genos.matrix == "1/0"] <- "1"
genos.matrix[genos.matrix == "0/0"] <- "2"
#print(genos.matrix[1:3,1:5])
write.csv(genos.matrix, file="genos_matrix.csv", row.names=T, col.names=T, append=F, quote=F)


#put in baypass format
#initilize matrix
afs=matrix(, nrow=dim(genos.filt)[2], ncol=length(levels(samples$PopulationName))*2)
row.names(afs)=loci_names
pops_double=vector(mode="character", length=length(levels(samples$PopulationName))*2)
#get population allele frequencies
#loop over populations
for (i in 1:length(levels(samples$PopulationName)))
    {
        print(levels(samples$PopulationName)[i])
        #subset for samples from the population
        samples_pop=subset(samples, PopulationName==levels(samples$PopulationName)[i], 
                                select=c(SampleID))
        print(dim(samples_pop)[1])
        #get genotypes and count alleles at each locus
        genos_pop=genos.filt[row.names(genos.filt) %in% samples_pop[,1],]
        loci=apply(genos_pop,2,paste,collapse="")
        ref_count=str_count(loci,"0")
        alt_count=str_count(loci,"1")
        na_count=str_count(loci,"[.]")

        #sanity check
        if (sum(is.na(match((ref_count+alt_count+na_count),dim(samples_pop)[1]*2)))>0)
            {print("warning--values do not add up for:")
             print(levels(samples$PopulationName)[i])
            }
        #print to matrix
        afs[,i*2-1]=ref_count
        afs[,i*2]=alt_count
        pops_double[i*2-1]=paste(levels(samples$PopulationName)[i],"_A",sep="")
        pops_double[i*2]=paste(levels(samples$PopulationName)[i],"_B",sep="")
    }

colnames(afs)=pops_double
write.csv(afs, file="pop_alleles_baypass.csv", row.names=T, col.names=T, append=F, quote=F)